In [ ]:
!pwd
!git clone https://github.com/fastai/fastbook.git
%cd fastbook
!pip install -r requirements.txt
!python3 -m pip install --upgrade Pillow

print("Setup Colab..")

!curl -s https://course.fast.ai/setup/colab | bash

In [6]:
#hide

from utils import *
from fastai2.vision.widgets import *
!pip install voila

key = '709b8f9a58ed4054a4ddb82f16b8c527'
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('content_url')
print(ims)

ImportError: cannot import name 'FileUpload' from 'ipywidgets' (/anaconda3/lib/python3.7/site-packages/ipywidgets/__init__.py)

In [ ]:
dest = 'images/grizzly.jpg'
download_url(ims[0], dest)

im = Image.open(dest)
im.to_thumb(128,128)


In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('content_url'))

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink);


In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
    dls = bears.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')



In [ ]:
learn_inf = load_learner(path/'export.pkl')
learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])


!jupyter serverextension enable voila —sys-prefix

